### About Keras layers

```python
layer.get_weights()
```
:returns the wiehgts of the layer as a list of Numpy arrays.

```python
later.set_weights(weights)
```
:sets the weights of the layer from a list of Numpy arrays (with the same shapes as the output of get_weights)

```python
layer.get_config()
```
:returns a dictionary containing the configuration of the layer. The layer can be reinstantiated from its config via:
```python
layer = Dense(32)
config = layer.get_config()
reconstructed_layer = Dense.from_config(config)
```
or
```python
from keras import layers
config = layer.get_config()
layer = layers.deserialized({'class_name': layer.__class__.__name__,
                             'config': config})
```

_____
### Dense

regular densely-connectted NN layer

```python
keras.layers.Dense(units, activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)
```

Dense imporments the operation: **output = activation(dot(input, kernel) + bias)**

- where activation is the element-wise activation function passed as the activation argument
- kernel is weights matrix created by the layer
- bias is a bias vector created vy the layer (only applicable if use_bias is True)

In [2]:
from keras.models import Sequential
from keras.layers import Dense

# as first layer in a sequential model:
model = Sequential()
model.add(Dense(32, input_shape=(16,)))
# now the model will take as input arrays of shape (*, 16)
# and output arrays of shape (*, 32)

# after the first layer, you don't need to specify
# the size of the input anymore:
model.add(Dense(32))

- Input shape

  - nD tensor with shape: (batch_size, ..., input_dim). The most common situation would be a 2D input with shape (batch_size, input_dim).

- Output shape

  - nD tensor with shape: (batch_size, ..., units). For instance, for a 2D input with shape  (batch_size, input_dim), the output would have shape (batch_size, units).

_____
### Activation

activation: name of activation function to use (softmax, elu, selu, softplus, softsign, relu, tanh, sigmoid, hard_sigmoid, linear), or alternatively, a Theano or TensorFlow operation.

- Input shape

  - Arbitrary. Use the keyword argument input_shape (tuple of integers, does not include the samples axis) when using this layer as the first layer in a model.

- Output shape

  - Same shape as input.

In [3]:
from keras.layers import Activation

Activation('relu')

_____
### Dropout

- Arguments

  - rate: float between 0 and 1. Fraction of the input units to drop.

  - noise_shape: 1D integer tensor representing the shape of the binary dropout mask that will be multiplied with the input. For instance, if your inputs have shape  (batch_size, timesteps, features) and you want the dropout mask to be the same for all timesteps, you can use noise_shape=(batch_size, 1, features).

  - seed: A Python integer to use as random seed.

In [4]:
from keras.layers import Dropout
Dropout(0.5, noise_shape=None, seed=None)

_____
### Flatten

- data_format: A string, one of channels_last (default) or channels_first. The ordering of the dimensions in the inputs. 

  - to preserve weight ordering when switching a model from one data format to another.
  - channels_last corresponds to inputs with shape  (batch, ..., channels)
  - channels_first corresponds to inputs with shape (batch, channels, ...)
  - It defaults to the image_data_format value found in your Keras config file at  ~/.keras/keras.json. If you never set it, then it will be "channels_last".

In [5]:
import sys
from keras.layers import Flatten
from keras.layers import Conv2D

model = Sequential()
model.add(Conv2D(64, (3, 3),
                 #border_mode='same',
                 input_shape=(3, 32, 32), padding='same'))
# now: model.output_shape == (None, 64, 32, 32)

model.add(Flatten())
# now: model.output_shape == (None, 65536)

keras documentation의 코드에는
```python
model = Sequential()
model.add(Conv2D(64, 3, 3,
                 border_mode='same',
                 input_shape=(3, 32, 32)))
'# now: model.output_shape == (None, 64, 32, 32)

model.add(Flatten())
'# now: model.output_shape == (None, 65536)
```
이라고 작성되어있음.

하지만 아래와 같은 오류메세지를 만나게됨.

```
~/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), input_shape=(3, 32, 32..., padding="same")`
  after removing the cwd from sys.path.
```

위의 코드와 같이 수정하면 됨! edit on gitHub을 시도해봄!

_____
### Input

- Arguments
  - shape: A shape tuple (integer), not including the batch size. For instance, shape=(32,) indicates that the expected input will be batches of 32-dimensional vectors.
  - batch_shape: A shape tuple (integer), including the batch size. For instance, batch_shape=(10, 32) indicates that the expected input will be batches of 10 32-dimensional vectors.  batch_shape=(None, 32) indicates batches of an arbitrary number of 32-dimensional vectors.
  - name: An optional name string for the layer. Should be unique in a model (do not reuse the same name twice). It will be autogenerated if it isn't provided.
  - dtype: The data type expected by the input, as a string (float32, float64, int32...)
  - sparse: A boolean specifying whether the placeholder to be created is sparse.
  - tensor: Optional existing tensor to wrap into the Input layer. If set, the layer will not create a placeholder tensor.
- Returns
  - A tensor.

In [42]:
from keras.models import Model
from keras.layers import Input

# this is a logistic regression in Keras
x = Input(shape=(32,))
y = Dense(16, activation='softmax')(x)
model = Model(x, y)

_____
### Reshape

Reshapes an output to a certain shape.

- Arguments
  - target_shape: target shape. Tuple of integers. Does not include the batch axis.
- Input shape
  - Arbitrary, although all dimensions in the input shaped must be fixed. Use the keyword argument input_shape (tuple of integers, does not include the batch axis) when using this layer as the first layer in a model.

- Output shape
  - (batch_size,) + target_shape
  
```python
from keras.layers import Reshape

# as first layer in a Sequential model
model = Sequential()
model.add(Reshape((3, 4), input_shape=(12,)))
# now: model.output_shape == (None, 3, 4)
# note: `None` is the batch dimension

# as intermediate layer in a Sequential model
model.add(Reshape((6, 2)))
# now: model.output_shape == (None, 6, 2)

# also supports shape inference using `-1` as dimension
model.add(Reshape((-1, 2, 2)))
# now: model.output_shape == (None, 3, 2, 2)
```

_____
### Permute

Permutes the dimensions of the input according to a given pattern.

Useful for e.g. connecting RNNs and convnets together.

- Arguments

  - dims: Tuple of integers. Permutation pattern, does not include the samples dimension. Indexing starts at 1. For instance,  (2, 1) permutes the first and second dimension of the input.

- Input shape

  - Arbitrary. Use the keyword argument input_shape (tuple of integers, does not include the samples axis) when using this layer as the first layer in a model.

- Output shape

  - Same as the input shape, but with the dimensions re-ordered according to the specified pattern.
  
```python
from keras.layers import Permute
model = Sequential()
model.add(Permute((2, 1), input_shape=(10, 64)))
# now: model.output_shape == (None, 64, 10)
# note: `None` is the batch dimension
```

_____
### ActivityRegularization

Layer that applies an update to the cost function based input activity.

- Arguments

  - l1: L1 regularization factor (positive float).
  - l2: L2 regularization factor (positive float).

- Input shape

  - Arbitrary. Use the keyword argument input_shape (tuple of integers, does not include the samples axis) when using this layer as the first layer in a model.

- Output shape

  - Same shape as input.


In [55]:
from keras.layers import ActivityRegularization
ActivityRegularization(l1=0.0, l2=0.0)